# Amazon Bedrock AgentCore Runtime での MCP サーバーホスティング - AWS IAM インバウンド認証

## 概要

このチュートリアルでは、MCP（Model Context Protocol）サーバーを Amazon Bedrock AgentCore Runtime でホスティングする方法を学びます。Amazon Bedrock AgentCore Python SDK を使用して、MCP ツールを Amazon Bedrock AgentCore 互換の MCP サーバーとしてラップします。

Amazon Bedrock AgentCore Python SDK は MCP サーバーの実装詳細を処理するため、ツールのコア機能に集中できます。コードを直接通信用の AgentCore 標準化 MCP プロトコル契約に変換します。

[MCP プロトコル](https://modelcontextprotocol.io/docs/getting-started/intro)仕様は従来、認証に OAuth トークンを必要としますが、AgentCore Runtime では MCP サーバーへのインバウンドリクエストに AWS IAM 認証情報を設定できるため、重要なエンタープライズ要件に対応します。

### チュートリアル詳細

| 情報             | 詳細                                                   |
|:-----------------|:------------------------------------------------------|
| チュートリアルタイプ | ツールホスティング                                      |
| ツールタイプ       | MCP サーバー                                            |
| チュートリアル構成  | AgentCore Runtime での MCP サーバーホスティング           |
| チュートリアル分野  | クロスバーティカル                                       |
| 例の複雑さ        | 簡単                                                    |
| 使用 SDK         | Amazon BedrockAgentCore Python SDK と MCP               |

### チュートリアルアーキテクチャ

このチュートリアルでは、MCP サーバーを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、`add_numbers`、`multiply_numbers`、`greet_user` の3つのツールを持つシンプルな MCP サーバーを使用します

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### チュートリアルの主な機能

* カスタムツールを備えた MCP サーバーの作成
* MCP サーバーのローカルテスト
* Amazon Bedrock AgentCore Runtime での MCP サーバーホスティング
* 認証を使用したデプロイ済み MCP サーバーの呼び出し

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報が設定済み
* Amazon Bedrock AgentCore SDK
* MCP（Model Context Protocol）ライブラリ
* Docker デーモンが実行中

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os

In [ ]:
boto_session = Session()
region = boto_session.region_name

agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_iam"

## MCP（Model Context Protocol）を理解する

MCP は AI モデルが外部データとツールに安全にアクセスできるようにするプロトコルです。主要な概念：

* **ツール**：AI がアクションを実行するために呼び出せる関数
* **Streamable HTTP**：AgentCore Runtime が使用するトランスポートプロトコル
* **セッション分離**：各クライアントは `Mcp-Session-Id` ヘッダーを介して分離されたセッションを取得
* **ステートレス操作**：サーバーはスケーラビリティのためにステートレス操作をサポートする必要があります

AgentCore Runtime は MCP サーバーがデフォルトパスとして `0.0.0.0:8000/mcp` でホストされることを期待します。

### プロジェクト構造

適切な構造でプロジェクトをセットアップしましょう：

```
mcp_server_project/
├── mcp_server.py              # メイン MCP サーバーコード
├── mcp_client.py          # ローカルテストクライアント
├── mcp_client_remote.py   # リモートテストクライアント
├── requirements.txt          # 依存関係
└── __init__.py              # Python パッケージマーカー
```

## MCP サーバーの作成

それでは、3つのシンプルなツールを備えた MCP サーバーを作成しましょう。サーバーは AgentCore Runtime 互換性のために必要な `stateless_http=True` で FastMCP を使用します。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """2つの数字を足し合わせる"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """2つの数字を掛け合わせる"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """ユーザーに名前で挨拶する"""
    return f"こんにちは、{name}さん！お会いできて嬉しいです。"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### このコードが行うこと

* **FastMCP**：ツールをホストできる MCP サーバーを作成
* **@mcp.tool()**：Python 関数を MCP ツールに変換するデコレーター
* **stateless_http=True**：AgentCore Runtime 互換性のために必要
* **ツール**：異なるタイプの操作を示す3つのシンプルなツール

## ローカルテストクライアントの作成

AgentCore Runtime にデプロイする前に、MCP サーバーをローカルでテストするためのクライアントを作成しましょう：

In [ ]:
%%writefile mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("利用可能なツール:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

 ### ローカルでテスト

MCP サーバーをローカルでテストするには：

1. **ターミナル1**：MCP サーバーを起動
   ```bash
   python mcp_server.py
   ```
   
2. **ターミナル2**：テストクライアントを実行
   ```bash
   python mcp_client.py
   ```

出力に3つのツールが一覧表示されるはずです。

## AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
print(f"使用するAWSリージョン: {region}")

required_files = ["mcp_server.py", "requirements.txt"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"必要なファイル {file} が見つかりません")
print("必要なファイルがすべて見つかりました ✓")

agentcore_runtime = Runtime()

print("AgentCore Runtimeを設定中...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name=tool_name,
)
print("設定完了 ✓")

## MCP サーバーを AgentCore Runtime に起動

Docker ファイルができたので、MCP サーバーを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("MCPサーバーをAgentCore Runtimeに起動中...")
print("数分かかる場合があります...")
launch_result = agentcore_runtime.launch()
print("起動完了 ✓")
print(f"エージェントARN: {launch_result.agent_arn}")
print(f"エージェントID: {launch_result.agent_id}")

In [ ]:
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime_iam/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server with inbound auth',
    Overwrite=True
)
print("✓ エージェントARNをParameter Storeに保存しました")

print("\n設定の保存が完了しました!")
print(f"エージェントARN: {launch_result.agent_arn}")

## リモートテストクライアントの作成

それでは、デプロイ済みの MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイ済みサーバーに接続します：

In [ ]:
%%writefile mcp_client_remote.py       
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    AWS SigV4 認証を備えた streamable HTTP トランスポートを作成。

    この関数は、AWS Signature Version 4 (SigV4) を使用してリクエストを認証する
    MCP クライアントトランスポートを作成します。標準の MCP クライアントは
    ネイティブで AWS IAM 認証をサポートしていないため、このギャップを埋めるために必要です。

    Args:
        mcp_url (str): MCP ゲートウェイエンドポイントの URL
        service_name (str): SigV4 署名用の AWS サービス名（通常 "bedrock-agentcore"）
        region (str): Gateway がデプロイされている AWS リージョン

    Returns:
        StreamableHTTPTransportWithSigV4: SigV4 認証用に設定されたトランスポートインスタンス

    例:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # 現在の boto3 セッションから AWS 認証情報を取得
    # これらの認証情報は SigV4 でリクエストに署名するために使用されます
    session = boto3.Session()
    credentials = session.get_credentials()

    # SigV4 署名機能を備えたカスタムトランスポートを作成して返す
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    ページネーションを処理しながら、MCP クライアントからツールの完全なリストを取得。

    MCP サーバーはページネーションされたレスポンスでツールを返す場合があります。
    この関数はページネーションを自動的に処理し、利用可能なすべてのツールを
    単一のリストで返します。

    Args:
        client: MCP クライアントインスタンス（strands.tools.mcp.mcp_client.MCPClient から）

    Returns:
        list: MCP サーバーから利用可能なすべてのツールの完全なリスト

    例:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # すべてのページを取得するまでループ
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # 取得すべきページがまだあるか確認
        if tmp_tools.pagination_token is None:
            # ページなし - 完了
            more_tools = False
        else:
            # さらにページがある - 次のページを取得する準備
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
            mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCP セッションを初期化中...")
                await session.initialize()
                print("✓ MCP セッション初期化完了")

                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()

                print("\n📋 利用可能な MCP ツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   説明: {tool.description}")
                    if hasattr(tool, "inputSchema") and tool.inputSchema:
                        properties = tool.inputSchema.get("properties", {})
                        if properties:
                            print(f"   パラメータ: {list(properties.keys())}")
                    print()

                print(f"✅ MCP サーバーへの接続に成功しました！")
                print(f"{len(tool_result.tools)} 個のツールが利用可能です。")

    except Exception as e:
        print(f"❌ MCP サーバーへの接続エラー: {e}")
        import traceback

        print("\n🔍 完全なエラートレースバック:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())

## デプロイ済み MCP サーバーのテスト

リモートクライアントを使用してデプロイ済み MCP サーバーをテストしましょう：

In [ ]:
print("デプロイされたMCPサーバーをテスト中...")
print("=" * 50)
!python mcp_client_remote.py

### MCP ツールのリモート呼び出し

それでは、ツールのリスト表示だけでなく、MCP の完全な機能を示すためにツールを呼び出す拡張クライアントを作成しましょう：

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import sys
import os
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    AWS SigV4 認証を備えた streamable HTTP トランスポートを作成。

    この関数は、AWS Signature Version 4 (SigV4) を使用してリクエストを認証する
    MCP クライアントトランスポートを作成します。標準の MCP クライアントは
    ネイティブで AWS IAM 認証をサポートしていないため、このギャップを埋めるために必要です。

    Args:
        mcp_url (str): MCP ゲートウェイエンドポイントの URL
        service_name (str): SigV4 署名用の AWS サービス名（通常 "bedrock-agentcore"）
        region (str): Gateway がデプロイされている AWS リージョン

    Returns:
        StreamableHTTPTransportWithSigV4: SigV4 認証用に設定されたトランスポートインスタンス

    例:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # 現在の boto3 セッションから AWS 認証情報を取得
    # これらの認証情報は SigV4 でリクエストに署名するために使用されます
    session = boto3.Session()
    credentials = session.get_credentials()

    # SigV4 署名機能を備えたカスタムトランスポートを作成して返す
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    ページネーションを処理しながら、MCP クライアントからツールの完全なリストを取得。

    MCP サーバーはページネーションされたレスポンスでツールを返す場合があります。
    この関数はページネーションを自動的に処理し、利用可能なすべてのツールを
    単一のリストで返します。

    Args:
        client: MCP クライアントインスタンス（strands.tools.mcp.mcp_client.MCPClient から）

    Returns:
        list: MCP サーバーから利用可能なすべてのツールの完全なリスト

    例:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # すべてのページを取得するまでループ
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # 取得すべきページがまだあるか確認
        if tmp_tools.pagination_token is None:
            # ページなし - 完了
            more_tools = False
        else:
            # さらにページがある - 次のページを取得する準備
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCP セッションを初期化中...")
                await session.initialize()
                print("✓ MCP セッション初期化完了")

                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()

                print("\n📋 利用可能な MCP ツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")

                print("\n🧪 MCP ツールのテスト:")
                print("=" * 50)

                try:
                    print("\n➕ add_numbers(5, 3) をテスト中...")
                    add_result = await session.call_tool(
                        name="add_numbers", arguments={"a": 5, "b": 3}
                    )
                    print(f"   結果: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")

                try:
                    print("\n✖️  multiply_numbers(4, 7) をテスト中...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers", arguments={"a": 4, "b": 7}
                    )
                    print(f"   結果: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")

                try:
                    print("\n👋 greet_user('Alice') をテスト中...")
                    greet_result = await session.call_tool(
                        name="greet_user", arguments={"name": "Alice"}
                    )
                    print(f"   結果: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")

                print("\n✅ MCP ツールのテストが完了しました！")

    except Exception as e:
        print(f"❌ MCP サーバーへの接続エラー: {e}")
        import traceback

        print("\n🔍 完全なエラートレースバック:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())

## ツール呼び出しのテスト

MCP ツールを実際に呼び出してテストしましょう：

In [ ]:
print("MCPツール呼び出しをテスト中...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

MCP サーバーを AgentCore Runtime に正常にデプロイできたので、次のことができます：

1. **さらにツールを追加**：MCP サーバーに追加のツールを拡張
2. **カスタム認証**：AWS IAM インバウンド認証を実装
3. **統合**：他の AgentCore サービスと統合

## クリーンアップ（オプション）

このチュートリアルで作成したリソースをクリーンアップする場合は、以下のセルを実行してください：

In [ ]:
try:
    ssm_client.delete_parameter(Name='/mcp_server/runtime_iam/agent_arn')
    print("✓ Parameter Storeのパラメータを削除しました")
except ssm_client.exceptions.ParameterNotFound:
    print("ℹ️  Parameter Storeのパラメータが見つかりません")

In [ ]:
destroy_bedrock_agentcore(
    config_path=Path(".bedrock_agentcore.yaml"),
    agent_name=tool_name,
    delete_ecr_repo=True
)

# 🎉 おめでとうございます！

以下を正常に完了しました：

✅ **MCP サーバーを作成** - カスタムツール付き  
✅ **ローカルでテスト** - MCP クライアントを使用  
✅ **認証をセットアップ** - Amazon Cognito を使用  
✅ **AWS にデプロイ** - AgentCore Runtime を使用  
✅ **リモートで呼び出し** - 適切な認証を使用  
✅ **MCP の概念を学習** - ベストプラクティスを含む  

MCP サーバーは現在 Amazon Bedrock AgentCore Runtime で実行中であり、本番環境で使用する準備ができています！

## まとめ

このチュートリアルでは、以下を学びました：
- FastMCP を使用した MCP サーバーの構築
- AgentCore 互換性のためのステートレス HTTP トランスポートの設定
- AWS IAM インバウンド認証のセットアップ
- AWS での MCP サーバーのデプロイと管理
- ローカルおよびリモートでのテスト
- ツール呼び出しのための MCP クライアントの使用

デプロイ済みの MCP サーバーは、より大きな AI アプリケーションやワークフローに統合できるようになりました！